In [164]:
# import required libraries
import os
import sys
### add system path to get other library directories
sys.path[0] = os.path.join(os.path.abspath(''),'..')

import geopandas as gpd
import warnings
import pandas as pd
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
import contextily as cx
import zipfile
import glob
from lib import visualizations as viz
warnings.filterwarnings('ignore')
import zipfile
import gzip

In [174]:
merged_2017 = pd.read_csv("/Users/victoriakielb/broadbandequity/data/standard_dataframes/standard_merged_2017 (2).csv")

In [167]:
print(list(merged_2017.columns))

['City', 'tract', 'state', 'county', 'GEOID', 'geometry', 'Est_Total: TOTAL POPULATION_tct17', 'Est_Total: SEX BY AGE_tct17', 'Est_Total: Male: SEX BY AGE_tct17', 'Est_Total: Male: Under 5 years SEX BY AGE_tct17', 'Est_Total: Male: 5 to 9 years SEX BY AGE_tct17', 'Est_Total: Male: 10 to 14 years SEX BY AGE_tct17', 'Est_Total: Male: 15 to 17 years SEX BY AGE_tct17', 'Est_Total: Male: 18 and 19 years SEX BY AGE_tct17', 'Est_Total: Male: 20 years SEX BY AGE_tct17', 'Est_Total: Male: 21 years SEX BY AGE_tct17', 'Est_Total: Male: 22 to 24 years SEX BY AGE_tct17', 'Est_Total: Male: 25 to 29 years SEX BY AGE_tct17', 'Est_Total: Male: 30 to 34 years SEX BY AGE_tct17', 'Est_Total: Male: 35 to 39 years SEX BY AGE_tct17', 'Est_Total: Male: 40 to 44 years SEX BY AGE_tct17', 'Est_Total: Male: 45 to 49 years SEX BY AGE_tct17', 'Est_Total: Male: 50 to 54 years SEX BY AGE_tct17', 'Est_Total: Male: 55 to 59 years SEX BY AGE_tct17', 'Est_Total: Male: 60 and 61 years SEX BY AGE_tct17', 'Est_Total: Male: 

### Renaming Conventions
```
standard_merged = standard_merged.rename(columns = {
  # Health Outcomes
    'Arthritis among adults aged >=18 Years': 'health_arthritis', #take inverse
    'Cancer (excluding skin cancer) among adults aged >=18 Years': 'health_cancer', #take inverse
    'Chronic kidney disease among adults aged >=18 Years': 'health_KdnyDisease', #take inverse
    'Chronic obstructive pulmonary disease among adults aged >=18 Years': 'health_PulDisease', #take inverse
    'Coronary heart disease among adults aged >=18 Years': 'health_HrtDisease', #take inverse
    'Current asthma among adults aged >=18 Years': 'health_asthma', #take inverse
    'Diagnosed diabetes among adults aged >=18 Years':'health_diab', #take inverse
    'High blood pressure among adults aged >=18 Years': 'health_HighBldPrs', #take inverse
    'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years': 'health_CholHigh'#take inverse
    'Mental health not good for >=14 days among adults aged >=18 Years':'health_GoodMent', #take inverse
    'Physical health not good for >=14 days among adults aged >=18 Years':'health_GoodPhys', #take inverse
    'Stroke among adults aged >=18 Years': 'health_stroke', #take inverse
    
  # Unhealthy Behaviors
    'Binge drinking among adults aged >=18 Years': 'behav_drink', #take inverse
    'Current smoking among adults aged >=18 Years': 'behav_smoking', #take inverse
    'No leisure-time physical activity among adults aged >=18 Years': 'behav_NoPhysAct', #take inverse
    'Obesity among adults aged >=18 Years': 'behav_obesity', #take inverse
    
  #Preventative Care
    'Cholesterol screening among adults aged >=18 Years': 'prev_CholScreen',
    'Current lack of health insurance among adults aged 18–64 Years': 'prev_HlthInsur', #take inverse
    'Taking medicine for high blood pressure control among adults aged >=18 Years with high blood pressure': 'prev_BloodMed',
    'Visits to doctor for routine checkup within the past Year among adults aged >=18 Years': 'prev_DocVisit',
    
  #Internet connectivity
    'PERC Est_Total: With an Internet subscription: PRESENCE AND TYPES OF INTERNET SUBSCRIPTIONS IN HOUSEHOLD_tct17': 'inter_conrate'
    }
    )
```

In [146]:
# Check for infinite or NaN values in X and Y
print(np.any(np.isnan(merged_2017['RPL_THEMES'])))
print(np.any(np.isinf(merged_2017['RPL_THEMES'])))
print(np.any(np.isnan(merged_2017['inter_conrate'])))
print(np.any(np.isinf(merged_2017['inter_conrate'])))
print(np.any(np.isnan(merged_2017['health_GoodPhys'])))
print(np.any(np.isinf(merged_2017['health_GoodPhys'])))

False
False
True
False
False
False


In [147]:
merged_2017.dropna(subset=['inter_conrate'], inplace=True) 

In [153]:
y_vars = ['All teeth lost among adults aged >=65 Years',
'health_arthritis',
'behav_drink',
'health_cancer',
'prev_CholScreen',
'health_KdnyDisease',
'health_PulDisease',
'health_HrtDisease',
'health_asthma',
'prev_HlthInsur',
'behav_smoking',
'health_diab',
'prev_tests',
'health_HighBldPrs',
'health_CholHigh',
'prev_mammogram',
'health_GoodMent',
'behav_NoPhysAct',
'behav_obesity',
'prev_MenPrev',
'prev_WomPrev',
'prev_smear',
'health_GoodPhys',
'behav_sleep',
'health_stroke',
'prev_BloodMed',
'prev_DentVisits',
'prev_DocVisit',
'health_indx',
'health_indx_outcomes',
'health_indx_behav',
'health_indx_prev']

merged_2017.dropna(subset=y_vars, inplace=True)

In [150]:
import statsmodels.api as sm

# Specify the dependent and independent variables
X = merged_2017[['RPL_THEMES', 'inter_conrate']]
y = merged_2017['health_GoodPhys']

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the multivariate linear regression model using OLS
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        health_GoodPhys   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                 1.813e+04
Date:                Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                        16:03:39   Log-Likelihood:                 33415.
No. Observations:               14455   AIC:                        -6.682e+04
Df Residuals:                   14452   BIC:                        -6.680e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.6898      0.001    711.890

In [154]:
X = merged_2017[['RPL_THEMES', 'inter_conrate']]

for y in y_vars:
    y_data = merged_2017[y]
    X = sm.add_constant(X)
    model = sm.OLS(y_data, X).fit()
    print(f"Summary for dependent variable: {y}")
    print(model.summary())
    print("\n\n\n")

Summary for dependent variable: All teeth lost among adults aged >=65 Years
                                         OLS Regression Results                                        
Dep. Variable:     All teeth lost among adults aged >=65 Years   R-squared:                       0.733
Model:                                                     OLS   Adj. R-squared:                  0.733
Method:                                          Least Squares   F-statistic:                 1.981e+04
Date:                                         Wed, 10 May 2023   Prob (F-statistic):               0.00
Time:                                                 16:04:06   Log-Likelihood:                -43339.
No. Observations:                                        14455   AIC:                         8.668e+04
Df Residuals:                                            14452   BIC:                         8.671e+04
Df Model:                                                    2                              

                            OLS Regression Results                            
Dep. Variable:         prev_mammogram   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     57.93
Date:                Wed, 10 May 2023   Prob (F-statistic):           8.71e-26
Time:                        16:04:07   Log-Likelihood:                 29222.
No. Observations:               14455   AIC:                        -5.844e+04
Df Residuals:                   14452   BIC:                        -5.841e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.8067      0.001    622.912

In [155]:
"""
Calculates the R-square value for inter_conrate and RPL_THEMES against y_var (health outcome of interest).
Runs a for-loop for each of the y-vars. Partial correlations are arranged in descending order. 
"""

results = []

for y in y_vars:
    y_data = merged_2017[y].dropna()
    X = sm.add_constant(X)
    model = sm.OLS(y_data, X).fit()
    r_sq = model.rsquared
    results.append((y, r_sq))

results_sorted = sorted(results, key=lambda x: x[1], reverse=True)

for r in results_sorted:
    print(f"{r[0]}: R-squared = {r[1]}")

health_indx: R-squared = 0.7494244100718124
All teeth lost among adults aged >=65 Years: R-squared = 0.732706111232337
behav_NoPhysAct: R-squared = 0.7188070342194952
health_GoodPhys: R-squared = 0.7150535006890472
health_indx_behav: R-squared = 0.7075518743757949
prev_DentVisits: R-squared = 0.6649988651913528
health_diab: R-squared = 0.6531620549350652
health_GoodMent: R-squared = 0.6493591255292939
behav_smoking: R-squared = 0.6470345981962484
behav_obesity: R-squared = 0.6450222115203559
health_KdnyDisease: R-squared = 0.638444274429667
prev_WomPrev: R-squared = 0.611696976140897
health_stroke: R-squared = 0.6038090276950664
behav_sleep: R-squared = 0.574618872885441
health_PulDisease: R-squared = 0.5714074681511178
health_indx_outcomes: R-squared = 0.5574619309626094
health_HrtDisease: R-squared = 0.49720912093113556
prev_MenPrev: R-squared = 0.49364586676102795
prev_tests: R-squared = 0.4901594864484512
health_HighBldPrs: R-squared = 0.47695057008705233
health_asthma: R-squared =

In [156]:
"""
Calculates the partial correlation of inter_conrate and y_var (health outcome of interest)
while keeping RPL_THEMES fixed. Runs a for-loop for each of the y-vars. 
Partial correlations are arranged in descending order. 
"""

results = []
print("Partial correlations of inter_conrate and y_vars (RPL_THEMES fixed)")
print("\n")
for y_var in y_vars:
    # Define the variables
    x = merged_2017['inter_conrate'].dropna().values
    y = merged_2017[y_var].dropna().values
    z = merged_2017['RPL_THEMES'].dropna().values

    # Compute the correlation matrix
    corr_matrix = np.corrcoef(np.array([x, y, z]))

    # Compute the inverse of the correlation matrix
    inv_corr_matrix = np.linalg.inv(corr_matrix)

    # Compute the partial correlation
    partial_corr = -inv_corr_matrix[0,1] / np.sqrt(inv_corr_matrix[0,0] * inv_corr_matrix[1,1])
    
    # Append the results to the list of tuples
    results.append((y_var, partial_corr))

# Sort the list of tuples in descending order based on the partial correlation
results.sort(key=lambda x: x[1], reverse=True)

# Print the results
for result in results:
    print(f"{result[0]}: {result[1]}")
    print("\n")

Partial correlations of inter_conrate and y_vars (RPL_THEMES fixed)


health_indx: 0.865692073737738


behav_NoPhysAct: 0.847822394141628


health_GoodPhys: 0.8455238446100515


health_indx_behav: 0.8411487156921374


prev_DentVisits: 0.8154199074775328


health_diab: 0.808079668553438


health_GoodMent: 0.8058282203173192


behav_smoking: 0.8043837673829041


behav_obesity: 0.8031275387202685


health_KdnyDisease: 0.7986936880544389


prev_WomPrev: 0.7817735001899585


health_stroke: 0.7767584724813499


behav_sleep: 0.7579707402363968


health_PulDisease: 0.7557130607011243


health_indx_outcomes: 0.7464648503572381


health_HrtDisease: 0.7034846846165652


prev_MenPrev: 0.7025431297375047


prev_tests: 0.7000454127782966


health_HighBldPrs: 0.6905433517598112


health_asthma: 0.6878720796111113


health_indx_prev: 0.6049043231267948


prev_HlthInsur: 0.5847519103726624


health_arthritis: 0.5327953584924958


health_CholHigh: 0.4130178898202107


prev_CholScreen: 0.3093248334158688

### Normalizing Data

In [159]:
from sklearn.preprocessing import StandardScaler

# Define the columns to standardize
cols_to_standardize = ['inter_conrate', 'RPL_THEMES', 'All teeth lost among adults aged >=65 Years', 
                            'health_arthritis',
                            'behav_drink',
                            'health_cancer',
                            'prev_CholScreen',
                            'health_KdnyDisease',
                            'health_PulDisease',
                            'health_HrtDisease',
                            'health_asthma',
                            'prev_HlthInsur',
                            'behav_smoking',
                            'health_diab',
                            'prev_tests',
                            'health_HighBldPrs',
                            'health_CholHigh',
                            'prev_mammogram',
                            'health_GoodMent',
                            'behav_NoPhysAct',
                            'behav_obesity',
                            'prev_MenPrev',
                            'prev_WomPrev',
                            'prev_smear',
                            'health_GoodPhys',
                            'behav_sleep',
                            'health_stroke',
                            'prev_BloodMed',
                            'prev_DentVisits',
                            'prev_DocVisit',
                            'health_indx',
                            'health_indx_outcomes',
                            'health_indx_behav',
                            'health_indx_prev']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the data using the scaler
merged_2017[cols_to_standardize] = scaler.fit_transform(merged_2017[cols_to_standardize])


In [160]:
X = merged_2017[['RPL_THEMES', 'inter_conrate']]
y_vars = ['All teeth lost among adults aged >=65 Years',
            'health_arthritis',
            'behav_drink',
            'health_cancer',
            'prev_CholScreen',
            'health_KdnyDisease',
            'health_PulDisease',
            'health_HrtDisease',
            'health_asthma',
            'prev_HlthInsur',
            'behav_smoking',
            'health_diab',
            'prev_tests',
            'health_HighBldPrs',
            'health_CholHigh',
            'prev_mammogram',
            'health_GoodMent',
            'behav_NoPhysAct',
            'behav_obesity',
            'prev_MenPrev',
            'prev_WomPrev',
            'prev_smear',
            'health_GoodPhys',
            'behav_sleep',
            'health_stroke',
            'prev_BloodMed',
            'prev_DentVisits',
            'prev_DocVisit',
            'health_indx',
            'health_indx_outcomes',
            'health_indx_behav',
            'health_indx_prev']
for y in y_vars:
    y_data = merged_2017[y]
    # X = sm.add_constant(X)
    # no need for constant because they're all nearly zero with a p-value of 1
    model = sm.OLS(y_data, X).fit()
    print(f"Summary for dependent variable: {y}")
    print(model.summary())
    print("\n\n\n")

Summary for dependent variable: All teeth lost among adults aged >=65 Years
                                             OLS Regression Results                                             
Dep. Variable:     All teeth lost among adults aged >=65 Years   R-squared (uncentered):                   0.733
Model:                                                     OLS   Adj. R-squared (uncentered):              0.733
Method:                                          Least Squares   F-statistic:                          1.981e+04
Date:                                         Wed, 10 May 2023   Prob (F-statistic):                        0.00
Time:                                                 16:05:43   Log-Likelihood:                         -10975.
No. Observations:                                        14455   AIC:                                  2.195e+04
Df Residuals:                                            14453   BIC:                                  2.197e+04
Df Model:           

Summary for dependent variable: prev_smear
                                 OLS Regression Results                                
Dep. Variable:             prev_smear   R-squared (uncentered):                   0.019
Model:                            OLS   Adj. R-squared (uncentered):              0.019
Method:                 Least Squares   F-statistic:                              141.0
Date:                Wed, 10 May 2023   Prob (F-statistic):                    2.24e-61
Time:                        16:05:43   Log-Likelihood:                         -20371.
No. Observations:               14455   AIC:                                  4.075e+04
Df Residuals:                   14453   BIC:                                  4.076e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.

In [128]:
results = []

for y in y_vars:
    y_data = merged_2017[y].dropna()
    X = sm.add_constant(X)
    model = sm.OLS(y_data, X).fit()
    r_sq = model.rsquared
    results.append((y, r_sq))

results_sorted = sorted(results, key=lambda x: x[1], reverse=True)

for r in results_sorted:
    print(f"{r[0]}: R-squared = {r[1]}")

Visits to dentist or dental clinic among adults aged >=18 Years: R-squared = 0.8288800734240348
All teeth lost among adults aged >=65 Years: R-squared = 0.7762648919131734
health_GoodPhys: R-squared = 0.7669972708196424
behav_NoPhysAct: R-squared = 0.7568245249290556
Older adult women aged >=65 Years who are up to date on a core set of clinical preventive services: Flu shot past Year, PPV shot ever, Colorectal cancer screening, and Mammogram past 2 Years: R-squared = 0.7292720842213587
health_GoodMent: R-squared = 0.722645406448846
Older adult men aged >=65 Years who are up to date on a core set of clinical preventive services: Flu shot past Year, PPV shot ever, Colorectal cancer screening: R-squared = 0.6876110696504124
health_diab: R-squared = 0.6756683559683857
Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50–75 Years: R-squared = 0.6725247827594001
behav_smoking: R-squared = 0.6624889542340627
health_KdnyDisease: R-squared = 0.6557521484818853
behav_obesi

In [130]:
# code that prints just the coefficients for RPL_themes and inter_conrate
X = merged_2017[['RPL_THEMES', 'inter_conrate']]

for y in y_vars:
    y_data = merged_2017[y]
    model = sm.OLS(y_data, X).fit()
    print(f"Coefficients for dependent variable: {y}")
    print(f"RPL_THEMES: {model.params['RPL_THEMES']:.4f}")
    print(f"inter_conrate: {model.params['inter_conrate']:.4f}")
    print("\n")

Coefficients for dependent variable: All teeth lost among adults aged >=65 Years
RPL_THEMES: 0.3073
inter_conrate: -0.6267


Coefficients for dependent variable: health_arthritis
RPL_THEMES: 0.2223
inter_conrate: 0.6991


Coefficients for dependent variable: behav_drink
RPL_THEMES: 0.3291
inter_conrate: -0.3438


Coefficients for dependent variable: health_cancer
RPL_THEMES: 0.4435
inter_conrate: 0.2008


Coefficients for dependent variable: prev_CholScreen
RPL_THEMES: -0.4800
inter_conrate: -0.0471


Coefficients for dependent variable: health_KdnyDisease
RPL_THEMES: -0.1888
inter_conrate: 0.6584


Coefficients for dependent variable: health_PulDisease
RPL_THEMES: -0.0257
inter_conrate: 0.7379


Coefficients for dependent variable: health_HrtDisease
RPL_THEMES: -0.0183
inter_conrate: 0.6901


Coefficients for dependent variable: health_asthma
RPL_THEMES: -0.0401
inter_conrate: 0.6593


Coefficients for dependent variable: prev_HlthInsur
RPL_THEMES: -0.5353
inter_conrate: 0.1838


Coef

In [163]:
# print the variables where ABSOLUTE coefficient of RPL_THEMES is greater than inter_conrate
# RPL_THEMES has bigger impact 

for y in y_vars:
    y_data = merged_2017[y]
    model = sm.OLS(y_data, X).fit()
    if abs(model.params['RPL_THEMES']) > abs(model.params['inter_conrate']):
        print(f"{y}")
        print(f"RPL_THEMES: {model.params['RPL_THEMES']:.4f}")
        print(f"inter_conrate: {model.params['inter_conrate']:.4f}")
        print("\n")